In [ ]:
# Статистическое сопоставление цены биткоина с ценами на нефть, драг металлы
# и одним из основных индексов развития мировой экономики (S&P500).

In [2]:
import pathlib
pathlib.Path().resolve()

WindowsPath('C:/Windows/System32')

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [17]:
df = pd.read_csv('SP500 oil gold bitcoin.csv', encoding='utf-8')
df

,Date,BITCOIN,Brent Oil,Crude Oil WTI,Gold,S&P500
0,2010-07-18,0.1,75.37,76.01,1188.0,1064.88
1,2010-07-19,0.1,75.62,76.54,1181.7,1071.25
2,2010-07-20,0.1,76.22,77.44,1191.5,1083.48
3,2010-07-21,0.1,75.37,76.56,1191.6,1069.59
4,2010-07-22,0.1,77.82,79.30,1195.5,1093.67
...,...,...,...,...,...,...
4349,2022-06-14,22157.3,121.17,118.93,1813.5,3735.48
4350,2022-06-15,22577.9,118.51,115.31,1819.6,3789.99
4351,2022-06-16,20386.6,119.81,117.59,1849.9,3666.77
4352,2022-06-17,20444.6,113.12,109.56,1840.6,3674.84


In [18]:
# Brent Oil - нефть марки Brent
# Crude Oil WTI - нефть марки WTI
# Gold - цена за тройскую унцию
# S&P500 - фондовый индекс, в корзину которого включено 505 акций компаний с самой большой капитализацией 

In [20]:
factors, x_test, bitcoin, y_test = train_test_split(df[['Brent Oil','Crude Oil WTI','Gold','S&P500']], df['BITCOIN']
, test_size=0.2)

In [21]:
bitcoin=bitcoin.values.reshape(-1,1)
factors=factors.values
factors.shape

(3483, 4)

In [22]:
ones = np.ones((factors.shape[0], 1))
factors = np.hstack((ones, factors))

XTX = factors.T.dot(factors)
np.linalg.det(XTX)

3.2418567519256204e+32

In [23]:
# Определитель не близок к нулю, поэтому можно записать аналитическое решение:

In [24]:
XTX_inv = np.linalg.inv(XTX)

b = XTX_inv.dot(factors.T).dot(bitcoin)

print(b)

[[-4.64835413e+04]
 [ 4.97958376e+01]
 [ 7.85678522e+01]
 [ 8.81709971e+00]
 [ 1.35516865e+01]]


In [ ]:
# Итак, наша модель регрессии имеет вид:

# y = -46483.5413 + 49.7958376*x_1 + 78.5678522*x_2 + 8.81709971*x_3 + 13.5516865*x_4,

# где x_1 - Brent Oil, x_2 - Crude Oil WTI, x_3 - Gold, x_4 - S&P500

In [29]:
bitcoin_pred = -46483.5413 + 49.7958376*x_test['Brent Oil'] + 78.5678522*x_test['Crude Oil WTI'] + 8.81709971*x_test['Gold'] + 13.5516865*x_test['S&P500']

In [30]:
R1 = bitcoin_pred.var() / bitcoin.var()
R1

0.878094751299001

In [ ]:
# Коэффициент детерминации, равный 0.878094751299001, означает, что 88% дисперсии переменной bitcoin учитывается 
# или объясняется моделью. Оставшиеся 12% обусловлены какими-то ещё факторами, которые модель не учла.

In [ ]:
# Оценка значимости уравнения линейной регрессии

In [33]:
k1 = 4
k2 = 3483 - 4 -1

T1 = (R1 / (1 - R1)) * (k2 / k1)
T1

6263.088705287429

In [34]:
from scipy import stats

alpha = 0.05

F_crit = stats.f.ppf(1 - alpha, k1, k2)
F_crit

2.3744870374556784

In [35]:
# Имеем T1 > F_crit, поэтому уравнение регрессии статистически значимо.